In [1]:
# libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

import re

In [2]:
# PANDAS OPTIONS
# Set maximum number of columns and rows to display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set the maximum column width to a high value
pd.set_option('display.max_colwidth', 1000)

In [3]:
# Define the URL to scrape
base_url = 'https://www.naturabuy.fr/Munitions-Balles-22LR-cat-884.html'
page_number = 1

# Create an empty list to store the scraped data
data = []

# Loop through the first two pages of the website
while page_number <= 2:

    # Construct the URL for the current page
    url = base_url + f'?PAGE={page_number}'

    # Make a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the item cards on the page
    cards = soup.find_all('a', class_='itemcard')

    # Loop through the item cards and scrape the information
    for card in cards[:5]:

        # Get the href attribute of the item card and construct the URL for the product page
        product_url = 'https://www.naturabuy.fr/' + card['href'].lstrip('/')

        # Make a GET request to the product page
        response = requests.get(product_url)

        # Parse the HTML content of the response using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        ##### Scrape the product name from the title tag
        try:
            product_name = soup.find('title').text.strip()
        except:
            product_name = 'N/A'
            
       # Scrape the manufacturer
        try:
            manufacturer_element = soup.select_one("html:-soup-contains('Marque :') body div#contall div#body_container div#body_container_in div#PAGE div#Columns div#mainProduct div#productWrapper div#blocGallery div#productCriteres div.critere div.criterevalue")
            if manufacturer_element:
                manufacturer = manufacturer_element.text.strip().replace("Marque :", "")
            else:
                manufacturer = "N/A"
        except:
            manufacturer = "N/A"
     
        # Scrape whether the item is new or used
        try:
            item_is_new = soup.find('span', id='availabilityCondition').text.strip()
        except:
            item_is_new = 'N/A'

        # Scrape the price
        try:
            price = soup.find('div', id='priceContainer').text.strip()
        except:
            price = 'N/A'

        # Scrape the shipping cost
        try:
            shipping_cost = soup.find('div', id='shippingsContainer').find('b').text.strip()
        except:
            shipping_cost = 'N/A'
            
        # Scrape product description
        try:
            product_description = soup.select_one('div#contall div#body_container div#body_container_in div#PAGE div#Columns div#Description').text.strip()
            # Remove '\n' and '\xa0'
            product_description = product_description.replace('\n', ' ').replace('\xa0', ' ')
        except:
            product_description = 'N/A'

        # Add the scraped data to the list
        data.append({
            'product_name': product_name,
            'product_link': product_url,
            'manufacturer': manufacturer,
            'is_new': item_is_new,
            'price': price,
            'shipping_cost': shipping_cost,
            'product_description': product_description
        })

        # Wait for a short time to avoid getting blocked
        time.sleep(1)

    # Increment the page number
    page_number += 1

# Convert the list of dictionaries to a pandas DataFrame and save it to a CSV file
df = pd.DataFrame(data)

#df

In [4]:
# change dtypes of columns for easier manipulation
df['product_name'] = df['product_name'].astype(str)
df['manufacturer'] = df['manufacturer'].astype(str)
df['is_new'] = df['is_new'].astype(str)
df['price'] = df['price'].astype(str)
df['shipping_cost'] = df['shipping_cost'].astype(str)

# change formatting of prices, remove currency, set as float
df['price'] = df['price'].str.replace(',', '.').str.extract('(\d+\.\d+)', expand=False).astype(float)
df['shipping_cost'] = df['shipping_cost'].str.replace(',', '.').str.extract('(\d+\.\d+)', expand=False).fillna(0).astype(float)

# change string values for new-used to binary
df["is_new"] = df["is_new"].map({"Neuf": 1, "Occasion": 0})

# add new column for Total price
df['total_price'] = df['price'] + df['shipping_cost']

# remove text from description that doesnt belong to the item itself, eg share buttons and shop category
df['product_description'] = df['product_description'].apply(lambda x: x.split("Flobert > Munitions - Balles 22LR")[1].strip())

#df

In [5]:
# build a list of 22LR ammo manufacturers

# manually built list instead of dynamically scraping each site.
# Website-agnostic approach. Increase in speed and decrease in scraping load.
# Missing brands can be found in df.manufacturer and entered here.

list_manufacturers = [
    'Aguila Ammunition',  # aguila is same as aquila
    'Aquila',  # aguila is same as aquila
    'American Eagle',
    'Armscor',
    'Australian Outback Ammo',
    'Barnaul',
    'Blaser',
    'Blazer',
    'Browning',
    'Cartoucherie Française',
    'CCI',
    'CBC',
    'Divers',
    'Eley',
    'ELD Performance',
    'Federal',  # Federal Premium and Federal are the same
    'Fiocchi',
    'Flobert',
    'Geco',
    'Gemtech',
    'Gevelot',
    'Golden Eagle',
    'Hornady',
    'Lapua',
    'Les Baer Custom',
    'Lot Diverses Marques',
    'Magtech',
    'Manufrance',
    'Mauser',
    'MaxxTech',
    'NCS',
    'Norma',
    'PMC',
    'PPU',
    'Rangemaster',
    'Remington',
    'RWS',
    'Sellier and Bellot',  # Sellier & Bellot and Sellier and Bellot are the same
    'SFM',
    'SK',
    'Solognac',
    'Spartan',
    'Speer',
    'Topshot',
    'Victory',
    'Winchester',
    'Wolf'
]

# function to search for manufacturer name in text using regex
def search_manufacturer(text):
    #pattern = '|'.join(list_manufacturers)
    pattern = '|'.join([re.escape(x) for x in list_manufacturers])
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group()
    else:
        return None

# apply search_manufacturer function to the product_name column
df['manufacturer'] = df.apply(lambda x: search_manufacturer(x['product_name']) if pd.isna(x['manufacturer']) or x['manufacturer'] == 'N/A' else x['manufacturer'], axis=1)

In [ ]:
#### create a regex pattern to match manufacturer names from the list
###manufacturers_pattern = re.compile(r"\b(" + "|".join(list_manufacturers) + r")\b")
###
#### extract manufacturer from product name or description
###def extract_manufacturer(text):
###    # try to extract from product name
###    match = manufacturers_pattern.search(text)
###    if match:
###        return match.group(1)
###    # if not found, try to extract from product description
###    else:
###        match = manufacturers_pattern.search(df.loc[df['product_name']==text, 'product_description'].values[0])
###        if match:
###            return match.group(1)
###        # if still not found, return None
###        else:
###            return None
###
#### apply function to extract manufacturer from product name or description
###df['manufacturer'] = df['product_name'].apply(extract_manufacturer)
###
#### check for empty cells, if any do a pass of regex on product description
###df.loc[df['manufacturer'].isnull(), 'manufacturer'] = df['product_description'].apply(extract_manufacturer)
###
#### if still no data, we fill with N/A
###df['manufacturer'].fillna('N/A', inplace=True)

In [ ]:
## regex to catch any number divisible by 50 (min qtty of rounds in a box of ammo)
#def extract_bullet_qtty(text):
#    # match any number that is divisible by 50 without remainder
#    regex = r"\b(0|[5-9]\d*[0]|100)\s*(?:boites de\s*)?(?:cartouches|balles|munitions)\b"
#    match = re.search(regex, text, re.IGNORECASE)
#    if match:
#        # extract the matched number and convert it to integer
#        qtty = int(match.group(1))
#        # round the quantity to the nearest 50
#        qtty = (qtty // 50) * 50
#        return qtty
#    else:
#        return None
#
## check titles with regex
#df['bullet_qtty'] = df['product_name'].apply(extract_bullet_qtty)
#
## check for empty cells, if any do a pass of regex on product description --- !!! DUPE avoidance !!!
#df.loc[df['bullet_qtty'].isnull(), 'bullet_qtty'] = df['product_description'].apply(extract_bullet_qtty)
#
## if still no data, we fill with 50 for default min number of ammo per box
#df['bullet_qtty'].fillna(50, inplace=True)

In [6]:
# regex to catch any number divisible by 50 (min qtty of rounds in a box of ammo)
def extract_bullet_qtty(text):
    # match any number that is divisible by 50 without remainder
    regex = r"\b(0|[5-9]\d*[0]|100)\s*(?:boites de\s*)?(?:cartouches|balles|munitions)\b|\bMunition \/ boite\s*:\s*(0|[5-9]\d*[0]|100)\b"
    match = re.search(regex, text, re.IGNORECASE)
    if match:
        # extract the matched number and convert it to integer
        qtty = int(match.group(1)) if match.group(1) else int(match.group(2))
        # round the quantity to the nearest 50
        qtty = (qtty // 50) * 50
        return qtty
    else:
        return None

# check titles with regex
df['bullet_qtty'] = df['product_name'].apply(extract_bullet_qtty)

# check for empty cells, if any do a pass of regex on product description --- !!! DUPE avoidance !!!
df.loc[df['bullet_qtty'].isnull(), df.columns[df.columns.get_loc('bullet_qtty')]] = df['product_description'].apply(extract_bullet_qtty)

# if still no data, we fill with 50 for default min number of ammo per box
df['bullet_qtty'].fillna(50, inplace=True)

#df.sort_values('bullet_qtty', ascending=False)

In [7]:
# calculate cost of individual bullet from all data
df["price_per_bullet"] = df["total_price"] / df["bullet_qtty"]

df

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,bullet_qtty,price_per_bullet
0,MUNITIONS SUBSONIC CAL. 22 LR boite de 50 jg84 - Munitions - Balles 22LR (9416266),https://www.naturabuy.fr/MUNITIONS-SUBSONIC-CAL-22-LR-boite-50-jg84-item-9416266.html,Winchester,1,18.90,8.6,Marque : WinchesterEtat de l'objet : NeufType : SubsoniqueType d'ogive : Creuse Ogives en plomb 42 grains. Profil Hollow Point (Pointe creuse) expansive,27.50,50.0,0.5500
1,Lot de 100 Sellier Bellot 22 LR short. - Munitions - Balles 22LR (10352362),https://www.naturabuy.fr/Lot-100-Sellier-Bellot-22-LR-short--item-10352362.html,Sellier & Bellot,1,26.20,7.2,Marque : Sellier & BellotEtat de l'objet : NeufType : Short - demi - courteType d'ogive : Plomb Lot 100 balles 22 lr Short 1.8 g Port revu si plusieurs achats. Colissimo si plusieurs lots. Voyez les autres annonces de la boutique. Licence ou permis obligatoire. Port revu si plusieurs achats,33.40,100.0,0.3340
2,100 Balles bosquette Flobert 6 mm - Munitions - Balles 22LR (10352361),https://www.naturabuy.fr/100-Balles-bosquette-Flobert-6-mm-item-10352361.html,Flobert,1,42.50,7.2,Marque : FlobertEtat de l'objet : NeufType : Bosquette Boite de 100 Munitions Flobert bosquette 6 mm court. Voyez les autres annonces de la boutique. Colissimo obligatoire pour plusieurs boites. A bientôt. Merci.,49.70,100.0,0.4970
3,( 22 Lr Aguila Super Extra HP par 50)Cartouches 22 LR Aguila Super Extra - pointe creuse cuivrée - Munitions - Balles 22LR (9010417),https://www.naturabuy.fr/-22-Lr-Aguila-Super-Extra-HP-par-50-Cartouches-22-LR-Aguila-Super-Extra-pointe-creuse-cuivree-item-9010417.html,Aguila,1,8.32,6.9,Marque : AguilaEtat de l'objet : NeufType : StandardType d'ogive : Creuse 22 LR AGUILA SUPER EXTRA HP PAR 50 UNIQUEMENTOgive cuivrée à profil Hollow Point (Pointe creuse) pour un faible encrassement du canon. Haute vélocité : 1280 fps (390 m/s) soit une énergie de 187 joules.Carton de 500 soit 10 boites de 50.Boites en 50 et suremballage par 2000 (40 boites),15.22,50.0,0.3044
4,SK FLATNOSE MATCH 22 LR - Munitions - Balles 22LR (8218347),https://www.naturabuy.fr/SK-FLATNOSE-MATCH-22-LR-item-8218347.html,SK,1,7.20,7.8,"Marque : SKEtat de l'objet : NeufType : Match MUNITIONS SK FLATNOSE MATCH 22 LR A l’inverse de leurs homologues Flatnose Basic, les munitions SK Flatnose Match seront à privilégier pour la précision en cible et permettront de réaliser des groupements serrés, le tout à un prix intéressant. Une munition particulièrement adaptée à un entrainement régulier ne souffrant d’aucune concession sur la qualité ni sur les performances. Type d’ogive Comme son nom l’indique, la SK Flatnose Match opte ici pour une ogive à tête plate de 40 grains qui compensera un certain de manque de pénétration par une plus grande déformation à l’impact. Un profil plus conciliant avec les infrastructures de tir et qui saura ménager leur durée de vie. Vitesse et énergie En sortie de bouche, la balle de 40 grains affiche une vitesse initiale de 328 m/s et développe 139 joules d’énergie cinétique. A 50 mètres, le projectile vole à une vitesse de 300 m/s et délivre 116 joules.",15.00,50.0,0.3000
5,Balles RWS - Cal. 22 / 6 mm flobert - Par 150 Par 3 22LR Ronde - Munitions - Balles 22LR (10290427),https://www.naturabuy.fr/Balles-RWS-Cal-22-6-mm-flobert-Par-150-Par-3-22LR-Ronde-item-10290427.html,RWS,1,119.39,6.9,"Etat de l'objet : Neuf Quantité : Par 3Calibre : 22LROgive : RondeRetrouvez un grand classique de la munition de petit calibre, à percussion annulaire et balle pointue. Approvisionnez ainsi vos carabines de jardin avec des projectiles de belle facture. - Ces munitions sont compatibles avec les carabines de jardin et calibres de tir sportif de calibre .22 Flobert et .22 LR. - Elles proposent un haut niveau de qualité typiquement RWS, avec une ogive plomb à tête pointue ou ronde (voir options) - Idéal pour le tir de loisir - Boîte de 150 balles Quantité: Par 1, par 3 ou par 5 (voir options)",

In [8]:
# Save updated DataFrame to CSV
df.to_csv('naturabuy_scraped_data.csv', index=False)

In [ ]:

# to do
# add scrape target - qtty of rounds. DONE
# cost per shot DONE
# product link DONE
# change is_new col data to 0 and 1 DONE

# df["QttyAmmo"] - > regex function to run over ProductName col. Also check product_description
# df["Cost_per_round"] = df["TotalPrice"] / df["QttyAmmo"]

# order of cols

#add to price selector:
#REGEX pattern - Munition / boite : 50